In [17]:
%pylab inline
import keras
import keras.backend as K
from keras import Input, Model, Sequential
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from matplotlib import pyplot as plt
import os, shutil

Populating the interactive namespace from numpy and matplotlib


/home/ing-luca/.local/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['concatenate']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [18]:
normalized_transactions_filepath = "../datasets/berka_dataset/usable/normalized_transactions_100.npy"

transactions = np.load(normalized_transactions_filepath)
np.random.shuffle(transactions)
N, D = transactions.shape
print(N, D)

53888 100


In [19]:
class MinibatchDiscrimination(Layer):
    """Concatenates to each sample information about how different the input
    features for that sample are from features of other samples in the same
    minibatch, as described in Salimans et. al. (2016). Useful for preventing
    GANs from collapsing to a single output. When using this layer, generated
    samples and reference samples should be in separate batches."""

    def __init__(self, nb_kernels, kernel_dim, init='glorot_uniform', weights=None,
                 W_regularizer=None, activity_regularizer=None,
                 W_constraint=None, input_dim=None, **kwargs):
        self.init = initializers.get(init)
        self.nb_kernels = nb_kernels
        self.kernel_dim = kernel_dim
        self.input_dim = input_dim

        self.W_regularizer = regularizers.get(W_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.W_constraint = constraints.get(W_constraint)

        self.initial_weights = weights
        self.input_spec = [InputSpec(ndim=2)]

        if self.input_dim:
            kwargs['input_shape'] = (self.input_dim,)
        super(MinibatchDiscrimination, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 2

        input_dim = input_shape[1]
        self.input_spec = [InputSpec(dtype=K.floatx(),
                                     shape=(None, input_dim))]

        self.W = self.add_weight(shape=(self.nb_kernels, input_dim, self.kernel_dim),
            initializer=self.init,
            name='kernel',
            regularizer=self.W_regularizer,
            trainable=True,
            constraint=self.W_constraint)

        # Set built to true.
        super(MinibatchDiscrimination, self).build(input_shape)

    def call(self, x, mask=None):
        activation = K.reshape(K.dot(x, self.W), (-1, self.nb_kernels, self.kernel_dim))
        diffs = K.expand_dims(activation, 3) - K.expand_dims(K.permute_dimensions(activation, [1, 2, 0]), 0)
        abs_diffs = K.sum(K.abs(diffs), axis=2)
        minibatch_features = K.sum(K.exp(-abs_diffs), axis=2)
#         return K.concatenate([x, minibatch_features], 1)
        return minibatch_features

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
#         return input_shape[0], input_shape[1]+self.nb_kernels
        return input_shape[0], self.nb_kernels

    def get_config(self):
        config = {'nb_kernels': self.nb_kernels,
                  'kernel_dim': self.kernel_dim,
#                   'init': self.init.__name__,
                  'W_regularizer': self.W_regularizer.get_config() if self.W_regularizer else None,
                  'activity_regularizer': self.activity_regularizer.get_config() if self.activity_regularizer else None,
                  'W_constraint': self.W_constraint.get_config() if self.W_constraint else None,
                  'input_dim': self.input_dim}
        base_config = super(MinibatchDiscrimination, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [56]:
class GAN:
    def __init__(self, timesteps, latent_dim):
        self._timesteps = timesteps
        self._latent_dim = latent_dim

    def build_model(self, lr):
        optimizer = RMSprop(lr)

        self._generator = self._build_generator(self._latent_dim, self._timesteps)
#         self._generator.compile(loss='binary_crossentropy', optimizer='rmsprop')

        self._discriminator = self._build_discriminator(self._timesteps)
        self._discriminator.compile(loss='binary_crossentropy', optimizer=RMSprop(0.00001))
        
        z = Input(shape=(self._latent_dim, ))
        fake = self._generator(z)

        for layer in self._discriminator.layers:
            layer.trainable = False
        self._discriminator.trainable = False

        valid = self._discriminator(fake)

        self._gan = Model(z, valid, 'GAN')

        self._gan.compile(
            loss='binary_crossentropy',
            optimizer=RMSprop(0.0001))

        self._gan.summary()
        self._generator.summary()
        self._discriminator.summary()
        return self._gan, self._generator, self._discriminator

    def _build_generator(self, noise_dim, timesteps):
        generator_inputs = Input((latent_dim, ))
        generated = generator_inputs
        
        generated = Lambda(lambda x: K.expand_dims(x))(generated)
        while generated.shape[1] < timesteps:
            generated = Conv1D(
                32, 3, activation='relu', padding='same')(generated)
            generated = UpSampling1D(2)(generated)
        generated = Conv1D(
            1, 3, activation='relu', padding='same')(generated)
        generated = Lambda(lambda x: K.squeeze(x, -1))(generated)
        generated = Dense(timesteps, activation='tanh')(generated)

        generator = Model(generator_inputs, generated, 'generator')

        return generator

    def _build_discriminator(self, timesteps):
        discriminator_inputs = Input((timesteps, ))
        discriminated = discriminator_inputs
        
        mbd = MinibatchDiscrimination(5, 3)(discriminated)
        mbd = Dense(1, activation='relu')(mbd)
        
        discriminated = Lambda(lambda x: K.expand_dims(x))(discriminated)
        while discriminated.shape[1] > 1:
            discriminated = Conv1D(32, 3, activation='relu', padding='same')(discriminated)
            discriminated = MaxPooling1D(2, padding='same')(discriminated)
        discriminated = Flatten()(discriminated)
        discriminated = Dense(32, activation='relu')(discriminated)
        discriminated = Dense(15, activation='relu')(discriminated)
        discriminated = Dense(1, activation='relu')(discriminated) 
        discriminated = Concatenate()([discriminated, mbd])
        discriminated = Dense(1, activation='sigmoid')(discriminated) 

        discriminator = Model(discriminator_inputs, discriminated, 'discriminator')
        return discriminator

    def train(self, batch_size, epochs, n_generator, n_discriminator, dataset,
              img_frequency):
        half_batch = int(batch_size / 2)

        losses = [[], []]
        for epoch in range(epochs):
            for _ in range(n_discriminator):
                indexes = np.random.randint(0, dataset.shape[0], half_batch)
                batch_transactions = dataset[indexes]

                noise = np.random.normal(0, 1, (half_batch, self._latent_dim))

                generated_transactions = self._generator.predict(noise)

                discriminator_loss_real = self._discriminator.train_on_batch(
                    batch_transactions, np.ones((half_batch, 1)))
                discriminator_loss_fake = self._discriminator.train_on_batch(
                    generated_transactions, np.zeros((half_batch, 1)))
                discriminator_loss = 0.5 * np.add(discriminator_loss_real,
                                                  discriminator_loss_fake)

            for _ in range(n_generator):
                noise = np.random.normal(0, 1, (batch_size, latent_dim))

                generator_loss = self._gan.train_on_batch(
                    noise, np.ones((batch_size, 1)))

            losses[0].append(generator_loss)
            losses[1].append(discriminator_loss)

            print("%d [D loss: %f] [G loss: %f]" % (epoch, discriminator_loss,
                                                    generator_loss))

            if epoch % img_frequency == 0:
                self._save_imgs(epoch)
                self._save_losses(losses)

    def _save_imgs(self, epoch):
        rows, columns = 5, 5
        noise = np.random.normal(0, 1, (rows * columns, latent_dim))
        generated_transactions = self._generator.predict(noise)

        plt.subplots(rows, columns, figsize=(15, 5))
        k = 1
        for i in range(rows):
            for j in range(columns):
                plt.subplot(rows, columns, k)
                plt.plot(generated_transactions[k - 1])
                plt.xticks([])
                plt.yticks([])
                plt.ylim(0, 1)
                k += 1
        plt.tight_layout()
        plt.savefig('gan/%05d.png' % epoch)
        plt.savefig('gan/last.png')
        plt.close()

    @staticmethod
    def _save_losses(losses):
        plt.plot(losses[0])
        plt.plot(losses[1])
        plt.legend(['generator', 'discriminator'])
        plt.savefig('gan/losses.png')
        plt.close()

In [60]:
timesteps = 100
batch_size = 64
epochs = int(1e5)
n_discriminator = 1
n_generator = 1
latent_dim = 2
lr = 0.00005
img_frequency = 250
timesteps = timesteps

In [61]:
if os.path.exists('gan'):
    shutil.rmtree('gan')
os.makedirs('gan')

In [62]:
gan = GAN(timesteps, latent_dim)
gan.build_model(lr)
gan.train(batch_size, epochs, n_generator, n_discriminator, transactions, img_frequency)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_49 (InputLayer)        (None, 2)                 0         
_________________________________________________________________
generator (Model)            (None, 100)               28645     
_________________________________________________________________
discriminator (Model)        (None, 1)                 21828     
Total params: 50,473
Trainable params: 28,645
Non-trainable params: 21,828
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_47 (InputLayer)        (None, 2)                 0         
_________________________________________________________________
lambda_39 (Lambda)           (None, 2, 1)              0         
_________________________________________________________________
c

/home/ing-luca/.local/lib/python3.5/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 8.115829] [G loss: 4.245936]
1 [D loss: 8.117163] [G loss: 4.191650]
2 [D loss: 7.886952] [G loss: 4.154172]
3 [D loss: 7.958153] [G loss: 4.124950]
4 [D loss: 8.122228] [G loss: 4.083753]
5 [D loss: 7.783841] [G loss: 4.072050]
6 [D loss: 7.652644] [G loss: 4.021445]
7 [D loss: 7.978312] [G loss: 3.956775]
8 [D loss: 7.884613] [G loss: 3.983658]
9 [D loss: 7.886579] [G loss: 3.854421]
10 [D loss: 7.658965] [G loss: 3.919488]
11 [D loss: 7.697742] [G loss: 3.782183]
12 [D loss: 7.896040] [G loss: 3.793311]
13 [D loss: 7.903032] [G loss: 3.728346]
14 [D loss: 7.900012] [G loss: 3.619027]
15 [D loss: 8.137096] [G loss: 3.548946]
16 [D loss: 7.995084] [G loss: 3.504474]
17 [D loss: 8.145336] [G loss: 3.456769]
18 [D loss: 8.140400] [G loss: 3.235947]
19 [D loss: 7.915621] [G loss: 3.355437]
20 [D loss: 7.831634] [G loss: 3.317924]
21 [D loss: 7.325203] [G loss: 3.050607]
22 [D loss: 7.717037] [G loss: 3.082629]
23 [D loss: 7.952381] [G loss: 2.962346]
24 [D loss: 7.772213] [G l

201 [D loss: 0.709485] [G loss: 0.695197]
202 [D loss: 0.774452] [G loss: 0.695431]
203 [D loss: 0.949939] [G loss: 0.695267]
204 [D loss: 0.833357] [G loss: 0.695220]
205 [D loss: 0.707921] [G loss: 0.694633]
206 [D loss: 0.709707] [G loss: 0.695339]
207 [D loss: 0.707737] [G loss: 0.695043]
208 [D loss: 0.708111] [G loss: 0.694945]
209 [D loss: 0.728167] [G loss: 0.694945]
210 [D loss: 0.908796] [G loss: 0.694757]
211 [D loss: 0.709076] [G loss: 0.741495]
212 [D loss: 0.710764] [G loss: 0.694877]
213 [D loss: 0.709513] [G loss: 0.694865]
214 [D loss: 0.709693] [G loss: 0.697498]
215 [D loss: 0.709177] [G loss: 0.695316]
216 [D loss: 0.709142] [G loss: 0.694456]
217 [D loss: 0.707636] [G loss: 0.695038]
218 [D loss: 0.709385] [G loss: 0.696004]
219 [D loss: 0.778611] [G loss: 0.695924]
220 [D loss: 1.128358] [G loss: 0.696327]
221 [D loss: 0.949130] [G loss: 0.695490]
222 [D loss: 0.709931] [G loss: 0.695129]
223 [D loss: 0.708718] [G loss: 0.695001]
224 [D loss: 0.709543] [G loss: 0.

KeyboardInterrupt: 